# Using the `pint.UnitRegistry.wraps` decorator to make functions unit-aware
Information about the decorator can be found in the [pint documentation](https://pint.readthedocs.io/en/stable/advanced/wrapping.html)

In [1]:
import pint
from pint import UnitRegistry
ureg = UnitRegistry()

### Non-unit-aware function

In [2]:
def add(a, b):
    # Add lengths a and b together. They must be the same units
    return a + b

In [3]:
add(1,2)

3

### Using the `wraps` decorator to make `add()` unit aware
`wrap` takes 3 arguments:
- ret: the return units (`None` will skip converting the result)
- args: the input units for each argument as an iterable (`None` will skip converting inputs)
- srict: if `True` all convertible arguments must be a Quantity (`True` by default)

Note that arugments to not turn into pint.Quantities. I'm not sure where/when the argument conversion is done.

In [4]:
@ureg.wraps(ret=ureg.meter, args=(ureg.meter, ureg.meter), strict=False)
def add(a, b):
    # Function to add two lengths together. `a` and `b` are assumed to be in meters unless stated otherwise. The return value is in meters.
    print(a, b)
    return a + b

In [5]:
# Plain number inputs
add(1,2)

1 2


3 <Unit('meter')>

In [6]:
# Quantity inputs using same UnitRegistry as the wraps decorator
a = ureg.Quantity(1, 'meter')
b = ureg.Quantity(2, 'meter')
add(a, b)

1 2


3 <Unit('meter')>

### Quantity inputs using a different UnitRegistry than the wraps decorator
This doesn't throw an error, but it does do something weird and unexpected. I'm not sure what's happening here, nor why the result has two sets of meter units. This is what we will run into if a user passes their own quantity objects into `geocat-comp` functions. It may be best to have them provide data in SI units and then we use `pint` under the hood to do any conversions we need.

In [7]:
# Quantity inputs using a different UnitRegistry than the wraps decorator
ureg_2 = UnitRegistry()  # Define a different unit registry
a = ureg_2.Quantity(1, 'meter')
b = ureg_2.Quantity(2, 'meter')
add(a, b)

1 meter 2 meter


3 <Unit('meter')> <Unit('meter')>

### Quantity inpus where `a` and `b` are from different UnitRegistry objects
This causes problems, which is expected as it is mentioned in the [pint documentation](https://pint.readthedocs.io/en/stable/getting/pint-in-your-projects.html). However, the error is different from the given example of using two different registries. Why? I'm not sure.

In [8]:
a = ureg.Quantity(1, 'meter')
b = ureg_2.Quantity(2, 'meter')
add(a, b)

1 2 meter


DimensionalityError: Cannot convert from 'meter' to 'dimensionless'